To fix:
- make sure the code isnt bullshit (since chatgpt etc)
- maybe use a better measure than pmi? (as in the original paper, yet example uses pmi)
- consider combining the rstb and rstl datasets (do many versions? one with all, 3 separate and one with rstl and rstb combined?)
- maybe make the freq-filter vary across years since some will have less (MAYBE)

TESTING CONTEXTS

In [1]:
import os
import pandas as pd

folder_path = "old/collocate_results_combined_ff5"

csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
combined_data = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

unique_words = combined_data['Word'].unique()
print(unique_words)

keyword = "well"
keyword_docs = []

for f in csv_files:
    df = pd.read_csv(f)
    if keyword in df['Word'].values:
        keyword_docs.append(f)

if keyword_docs:
    print(f"The keyword '{keyword}' is found in the following document(s):")
    for doc in keyword_docs:
        print(doc)
else:
    print(f"The keyword '{keyword}' is not found in any document.")

['matter' 'theſe' 'known' 'many' 'important' 'respecting' 'new'
 'following' 'state' 'case' 'however' 'prove' 'stated' 'experiment'
 'preceding' 'ascertained' 'appear' 'general' 'adduced' 'chemical'
 'series' 'may' 'number' 'observation' 'curious' 'observed' 'well'
 'appears' 'singular' 'importance' 'propriety' 'certainty' 'noticed'
 'principal' 'connected' 'interesting' 'proved' 'remarkable' 'might'
 'induction' 'accordance' 'recorded' 'knowledge' 'additional' 'foregoing'
 'mentioned' 'striking' 'shown' 'induced' 'already' 'point' 'described'
 'explanation' 'establish' 'penetration' 'supported' 'pointed' 'express'
 'established' 'existence' 'think' 'always' 'latter' 'system' 'made'
 'attention' 'mere' 'quantic' 'seems' 'founded' 'evidence' 'account'
 'evident' 'probable' 'explain' 'yet' 'due' 'anatomical' 'expression'
 'arises' 'conic' 'upon' 'owing' 'bearing' 'explained' 'interest' 'partly'
 'noted' 'statement' 'obseryed' 'confirmed' 'experimental' 'detailed'
 'interpretation' 'mind'

In [4]:
import pandas as pd
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import csv
import os
import re
from collections import defaultdict

def analyze_collocates(text_dirs, output_dir, word_of_interest, window_size=5, collocate_window=5, freq_filter=5, top_n=10):
    def extract_year_from_filename(filename):
        match = re.search(r'rst[bla]?_?(\d{4})', filename)
        return int(match.group(1)) if match else None

    def read_texts_by_year(directories):
        texts_by_year = defaultdict(list)
        print("Reading text files...")
        for directory in directories:
            for filename in os.listdir(directory):
                if filename.endswith('.txt'):
                    year = extract_year_from_filename(filename)
                    if year:
                        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                            text = file.read()
                        text = text.replace('ſ', 's').replace('Å¿', 's').replace('obseryed', 'observed').replace('thab', 'that')
                        texts_by_year[year].append(text)
        print("Text files read and organized by year.")
        return texts_by_year

    stop_words = set(stopwords.words('english')).union({'al', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten'})
    lemmatizer = WordNetLemmatizer()

    def process_tokens(text):
        tokens = word_tokenize(text.lower())
        return [lemmatizer.lemmatize(token) for token in tokens 
                if token.isalpha() and len(token) > 1 and token not in stop_words and token not in string.punctuation]

    def get_collocates_by_sliding_window(texts_by_year):
        collocates_per_window = {}
        all_years = sorted(texts_by_year.keys())
        start_year, end_year = min(all_years), max(all_years)

        print("Analyzing collocates by sliding window...")
        for window_start in range(start_year, end_year - window_size + 2):
            window_end = window_start + window_size - 1
            window_texts = [text for year in range(window_start, window_end + 1) 
                            for text in texts_by_year.get(year, [])]
            
            all_tokens = [token for text in window_texts for token in process_tokens(text)]
            
            total_tokens = len(all_tokens)
            total_texts = len(window_texts)
            
            finder = BigramCollocationFinder.from_words(all_tokens)
            finder.apply_freq_filter(freq_filter)
            
            collocations = finder.score_ngrams(BigramAssocMeasures().pmi)
            
            collocate_stats = []
            collocate_contexts = defaultdict(list)

            word_of_interest_lower = word_of_interest.lower()
            word_of_interest_freq = sum(finder.word_fd[word] for word in finder.word_fd if word.lower() == word_of_interest_lower)

            for bigram, pmi in collocations:
                if word_of_interest_lower in (word.lower() for word in bigram):
                    other_word = bigram[0] if bigram[1].lower() == word_of_interest_lower else bigram[1]
                    observed_freq = finder.ngram_fd[bigram]
                    word_freq = finder.word_fd[other_word]
                    expected_freq = (word_of_interest_freq * word_freq) / total_tokens
                    num_texts = sum(1 for text in window_texts if other_word.lower() in text.lower().split())
                    
                    collocate_stats.append({
                        'word': other_word,
                        'total_corpus': word_freq,
                        'expected_freq': expected_freq,
                        'observed_freq': observed_freq,
                        'num_texts': num_texts,
                        'pmi': pmi
                    })
                    
                    for text in window_texts:
                        words = text.split()
                        for i, word in enumerate(words):
                            if word.lower() == word_of_interest_lower and other_word.lower() in [w.lower() for w in words[max(0, i-collocate_window):i+collocate_window+1]]:
                                context = ' '.join(words[max(0, i-collocate_window):i+collocate_window+1])
                                collocate_contexts[other_word].append((f"Years {window_start}-{window_end}: {context}", other_word))
            
            collocate_stats.sort(key=lambda x: x['pmi'], reverse=True)
            top_collocates = collocate_stats[:top_n]
            
            filtered_contexts = [context for collocate in top_collocates for context in collocate_contexts.get(collocate['word'], [])]
            
            collocates_per_window[f"{window_start}-{window_end}"] = {
                'collocates': top_collocates,
                'total_tokens': total_tokens,
                'total_texts': total_texts,
                'contexts': filtered_contexts
            }
            print(f"Years {window_start}-{window_end}: {len(collocate_stats)} collocates found")

        return collocates_per_window

    print("Starting text processing...")
    texts_by_year = read_texts_by_year(text_dirs)
    collocates_by_window = get_collocates_by_sliding_window(texts_by_year)

    os.makedirs(output_dir, exist_ok=True)
    print("Saving results to CSV files...")

    for window, data in collocates_by_window.items():
        filename = os.path.join(output_dir, f'collocates_{window}.csv')
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['No.', 'Word', 'Total no. in window corpus', 'Expected collocate frequency',
                             'Observed collocate frequency', 'In no. of texts', 'Mutual Information value'])
            
            for i, collocate in enumerate(data['collocates'], 1):
                writer.writerow([
                    i,
                    collocate['word'],
                    collocate['total_corpus'],
                    f"{collocate['expected_freq']:.2f}",
                    collocate['observed_freq'],
                    collocate['num_texts'],
                    f"{collocate['pmi']:.2f}"
                ])

        context_file = os.path.join(output_dir, f'collocate_contexts_{window}.csv')
        with open(context_file, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Context', 'Collocate'])
            writer.writerows(data['contexts'])

    print(f"Collocate analysis completed. Results saved in the '{output_dir}' directory.")

analyze_collocates([r"D:\Fact_fiction_corpus\texts\royal society\txt_rstb", 
                    r"D:\Fact_fiction_corpus\texts\royal society\txt_rstl", 
                    r"D:\Fact_fiction_corpus\texts\royal society\txt_rsta"], 
                   "collocate_results_combined_fact", "fact")

analyze_collocates([r"D:\Fact_fiction_corpus\texts\royal society\txt_rstb", 
                    r"D:\Fact_fiction_corpus\texts\royal society\txt_rstl", 
                    r"D:\Fact_fiction_corpus\texts\royal society\txt_rsta"], 
                   "collocate_results_combined_fiction", "fiction")

Starting text processing...
Reading text files...
Text files read and organized by year.
Analyzing collocates by sliding window...
Years 1665-1669: 0 collocates found
Years 1666-1670: 0 collocates found
Years 1667-1671: 0 collocates found
Years 1668-1672: 0 collocates found
Years 1669-1673: 1 collocates found
Years 1670-1674: 1 collocates found
Years 1671-1675: 1 collocates found
Years 1672-1676: 1 collocates found
Years 1673-1677: 1 collocates found
Years 1674-1678: 0 collocates found
Years 1675-1679: 0 collocates found
Years 1676-1680: 0 collocates found
Years 1677-1681: 0 collocates found
Years 1678-1682: 0 collocates found
Years 1679-1683: 0 collocates found
Years 1680-1684: 0 collocates found
Years 1681-1685: 1 collocates found
Years 1682-1686: 1 collocates found
Years 1683-1687: 1 collocates found
Years 1684-1688: 1 collocates found
Years 1685-1689: 1 collocates found
Years 1686-1690: 0 collocates found
Years 1687-1691: 0 collocates found
Years 1688-1692: 0 collocates found
Years